<font  style="font-size: 4rem; color: darkviolet"> Convolutional Neural Networks in TensorFlow </font>

AA - 2022/23 - TP7

*This assignement is inspired by the Deep Learning course on Coursera by Andrew Ng, Stanford University, for which we are thankful.*

In this assignment, you will learn how to build and train Convolutional Neural Networks (ConvNets) using TensorFlow Keras API. You will be working on two different tasks. First, you will build a binary classifier using the Sequential API to classify the mood of a given person as either positive or negative. Next, you will build a multiclass classifier using the Functional API to identify sign language digits.

Before starting this assignment, you should have a basic understanding of TensorFlow.

You're encouraged to read the official documentation. You can find the docs for the Sequential and Functional APIs here:

https://www.tensorflow.org/guide/keras/sequential_model

https://www.tensorflow.org/guide/keras/functional


## Table of Contents
- [1 - The Sequential API](#1)
    - [1.1 - The Happy House Dataset](#1.1)
    - [1.2 - Create the Sequential Model](#1.2)
        - [Exercise 1.2.1 - happyModel](#ex-1.2.1)
    - [1.3 - Train and Evaluate the Model](#1.3)
- [2 - The Functional API](#2)
    - [2.1 - The SIGNS Dataset](#2.1)
    - [2.2 - Forward Propagation](#2.2)
        - [Exercise 2.2.1 - convolutional_model](#ex-2.2.1)
    - [2.3 - Train the Model](#2.3)
    - [2.4 - History](#2.4)
- [3 - Exercise - output volume](#ex-3) 

In [1]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
from data.cnn_utils import *
from data.test_utils import summary, comparator

%matplotlib inline
np.random.seed(1)

2023-04-12 07:29:54.773928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 07:29:54.913064: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-12 07:29:54.913085: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-12 07:29:55.685202: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# <a name='1'></a>
# <font color='darkviolet'> 1 - The Sequential API

When it comes to practical applications of deep learning, programming frameworks are essential tools that offer a wide range of built-in functions to help you create and train models with ease. One such framework is Keras, which is a high-level abstraction built on top of TensorFlow that provides even more simplified and optimized model building and training.

You'll be using Keras' Sequential API to create a deep learning model. This API allows you to build models layer by layer, and is particularly well-suited for simple tasks where each layer has exactly one input tensor and one output tensor. Using the Sequential API is straightforward and easy to understand, making it a great choice for beginners.

<a name='1.1'></a>
## <font color='darkviolet'> 1.1 - The Happy House dataset

The Happy House dataset comprises images of individuals' faces, and your objective will be to create a ConvNet that can accurately identify whether these individuals are smiling or not. This is an important task because only those who are smiling will be granted entry into the Happy House.

In [2]:
def load_happy_dataset():
    # Load the training dataset
    train_dataset = h5py.File('data/train_happy.h5', "r")
    
    # Extract the features and labels for the training set
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # labels

    # Load the test dataset
    test_dataset = h5py.File('data/test_happy.h5', "r")
    
    # Extract the features and labels for the test set
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # labels

    # Extract the list of classes
    classes = np.array(test_dataset["list_classes"][:]) 
    
    # Reshape the labels to match the expected shape
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    # Return the dataset as a tuple
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
# Load the Happy House dataset
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_happy_dataset()

# Normalize the image vectors by dividing by 255
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Reshape the labels to match the expected shape (m, 1), where m is the number of examples
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

In [4]:
# Print the shapes of the datasets and labels
print("Number of training examples = " + str(X_train.shape[0]))
print("Number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

Number of training examples = 600
Number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


#### <font color='blue'> Q1.1 Describe your dataset.

<a name='1.2'></a>
## <font color='darkviolet'> 1.2 - Create the Sequential Model 
    
The **TensorFlow Keras Sequential** API is useful for building simple models where layers are arranged in a sequential order. You can add and remove layers using the `.add()` and `.pop()` methods, respectively. A Sequential model can be thought of as a list of layers, where the order in which they are specified matters. 

In Keras, you must specify the input shape in advance for any layer construction, since the shape of the weights is based on the shape of the inputs. To create a Sequential model, pass a list of layers to the Sequential constructor.

<a name='ex-1.2.1'></a>
### <font color='blue'> Exercise 1.2.1 - happyModel

Implement the `happyModel` function below to build the following model: `ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> FLATTEN -> DENSE`. 
    
Take help from [tf.keras.layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers).

Plug in the following parameters for all the steps:

 - [ZeroPadding2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ZeroPadding2D): padding 3, input shape 64 x 64 x 3
 - [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D): Use 32 7x7 filters, stride 1
 - [BatchNormalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization): for axis 3
 - [ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU)
 - [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D): Using default parameters
 - [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) the previous output.
 - Fully-connected ([Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) layer: Apply a fully connected layer with 1 neuron and a sigmoid activation. 
 

In [5]:
def happyModel():
    """
    Implements the forward propagation for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> FLATTEN -> DENSE
    
    Arguments:
    None

    Returns:
    model -- TF Keras model (object containing the information for the entire training process) 
    """

    # Define the layers of the model using the Sequential API
    model = tf.keras.Sequential([
        
        # ZeroPadding2D with padding 3, input shape of 64 x 64 x 3
        #TODO

        
        # Conv2D with 32 7x7 filters and stride of 1
        #TODO

        
        # BatchNormalization for axis 3
        #TODO

        
        # ReLU
        #TODO

        
        # Max Pooling 2D with default parameters
        #TODO

        
        # Flatten layer
        #TODO

        
        # Dense layer with 1 unit & 'sigmoid' activation
        #TODO

    ])
    
    # Return the model
    return model


In [6]:
happy_model = happyModel()
# Print a summary for each layer
for layer in summary(happy_model):
    print(layer)
    
output = [['ZeroPadding2D', (None, 70, 70, 3), 0, ((3, 3), (3, 3))],
            ['Conv2D', (None, 64, 64, 32), 4736, 'valid', 'linear', 'GlorotUniform'],
            ['BatchNormalization', (None, 64, 64, 32), 128],
            ['ReLU', (None, 64, 64, 32), 0],
            ['MaxPooling2D', (None, 32, 32, 32), 0, (2, 2), (2, 2), 'valid'],
            ['Flatten', (None, 32768), 0],
            ['Dense', (None, 1), 32769, 'sigmoid']]
    
comparator(summary(happy_model), output)

All tests passed!


2023-04-12 07:29:56.584711: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-12 07:29:56.584777: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-12 07:29:56.584828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fgalassi): /proc/driver/nvidia/version does not exist
2023-04-12 07:29:56.585259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


After creating a deep learning model, the next step is to compile it for training by specifying an optimizer and loss function. In addition, you can also specify metrics such as accuracy to track during training.

In [7]:
happy_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [8]:
happy_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

#### <font color='blue'> Q1.2.1 What is the difference between trainable and non-trainable parameters in a deep learning model?

#### <font color='blue'> Q1.2.2 How to compute the number of parameters reported per layer? Can you identify the origin of the 64 non-trainable parameters?
        
#### <font color='blue'> Q1.2.3 You can note that the first output dimension of a layers is *None*. Can you explain why this is the case?

<a name='1.3'></a>
## <font color='darkviolet'> 1.3 - Train and Evaluate the Model

Once you've created and compiled your model, and verified its structure using `model.summary()`, it's time to start training. Simply call `.fit()` to begin the training process.

One of the advantages of using TensorFlow is that it automates many of the tedious and complex aspects of machine learning, such as backpropagation. You do have the option to specify the number of epochs and the size of the mini-batch. 

In [ ]:
happy_model.fit(X_train, Y_train, epochs=10, batch_size=32)

#### <font color='blue'> Q1.3.1 What does each output line indicate ?


After training your model using `.fit()`, you can evaluate its performance on your test set by calling `.evaluate()`. This function returns the value of the **loss function** and any **performance metrics** you specified during compilation. In this case, the loss function used was binary_crossentropy and the metric used was accuracy.

In [ ]:
happy_model.evaluate(X_test, Y_test)

<a name='2'></a>
# <font color='darkviolet'> 2 - The Functional API

The [Functional API](https://www.tensorflow.org/guide/keras/functional) in TensorFlow can handle more complex models than the Sequential API, allowing for shared layers, and layers with multiple inputs or outputs. 
One example of a more complex layer connection is a skip connection, which skips some layers in the network and feeds the output directly to a later layer in the network.
    
You will use Keras flexible Functional API to build a ConvNet that can differentiate between 6 sign language digits. 

<a name='2.1'></a>
## <font color='darkviolet'> 2.1 - The SIGNS Dataset

The SIGNS dataset is a collection of 6 signs representing numbers from 0 to 5.

In [ ]:
def load_signs_dataset():
    # Load the training dataset
    train_dataset = h5py.File('data/train_signs.h5', "r")
    
    # Extract the features and labels for the training set
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # labels

    # Load the test dataset
    test_dataset = h5py.File('data/test_signs.h5', "r")
    
    # Extract the features and labels for the test set
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # labels

    # Extract the list of classes
    classes = np.array(test_dataset["list_classes"][:]) 
    
    # Reshape the labels to match the expected shape
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    # Return the dataset as a tuple
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_signs_dataset()

In [ ]:
# Example of an image from the dataset
index = 9
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

In [ ]:
# Normalize the pixel values of the images by dividing each pixel value by 255
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert the target labels to one-hot vectors
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

#### <font color='blue'> Q2.1.1 Describe your dataset.

<a name='2.2'></a>
### <font color='darkviolet'> 2.2 - Forward Propagation

In TensorFlow, there are built-in functions that implement the convolution steps. By now, you should be familiar with how TensorFlow builds computational graphs. In the [Functional API](https://www.tensorflow.org/guide/keras/functional), you create a graph of layers. This is what allows such great flexibility.

However, the following model could also be defined using the Sequential API since the information flow is on a single line. Anyhw, we want you to learn to use the Functional API.

Begin building your graph of layers by creating an input node as a callable object:

- **input_img = tf.keras.Input(shape=input_shape):** 

Then, create a new node in the graph of layers by calling a layer on the `input_img` object: 

- **tf.keras.layers.Conv2D(filters= ... , kernel_size= ... , padding='same')(input_img):** Read the full documentation on [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D).

- **tf.keras.layers.MaxPool2D(pool_size=(f, f), strides=(s, s), padding='same'):** `MaxPool2D()` downsamples your input using a window of size (f, f) and strides of size (s, s) to carry out max pooling over each window.  For max pooling, you usually operate on a single example at a time and a single channel at a time. Read the full documentation on [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D).

- **tf.keras.layers.ReLU():** computes the element-wise ReLU of Z (which can be any shape). You can read the full documentation on [ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU).

- **tf.keras.layers.Flatten()**: given a tensor "P", this function takes each training (or test) example in the batch and flattens it into a 1D vector.  

    * If a tensor P has the shape (batch_size,h,w,c), it returns a flattened tensor with shape (batch_size, k), where $k=h \times w \times c$.  "k" equals the product of all the dimension sizes other than the first dimension.
    
    * For example, given a tensor with dimensions [100, 2, 3, 4], it flattens the tensor to be of shape [100, 24], where 24 = 2 * 3 * 4.  You can read the full documentation on [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten).

- **tf.keras.layers.Dense(units= ... , activation='softmax')(F):** given the flattened input F, it returns the output computed using a fully connected layer. You can read the full documentation on [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).

In the last function above (`tf.keras.layers.Dense()`), the fully connected layer automatically initializes weights in the graph and keeps on training them as you train the model. Hence, you did not need to initialize those weights when initializing the parameters.

Lastly, before creating the model, you'll need to define the output using the last of the function's compositions (in this example, a Dense layer): 

- **outputs = tf.keras.layers.Dense(units=6, activation='softmax')(F)** 


<a name='ex-2.2.1'></a>
### <font color='blue'> Exercise 2.2.1 - convolutional_model

Implement the `convolutional_model` function below to build the following model: `CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> DENSE`. Use the functions above! 

Also, plug in the following parameters for all the steps:

 - [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D): Use 8 filters 4 by 4 , stride 1, padding is "SAME"
 - [ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU)
 - [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D): Use an 8 by 8 filter size and an 8 by 8 stride, padding is "SAME"
 - **Conv2D**: Use 16 filters 2 by 2, stride 1, padding is "SAME"
 - **ReLU**
 - **MaxPool2D**: Use a 4 by 4 filter size and a 4 by 4 stride, padding is "SAME"
 - [Flatten](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten) the previous output.
 - Fully-connected ([Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) layer: apply a fully connected layer with 6 neurons and a softmax activation. 

In [ ]:
def convolutional_model(input_shape):
    """
    Implements a convolutional neural network model using Keras.

    Arguments:
    input_shape -- shape of the input data

    Returns:
    model -- Keras model instance
    """
    
    # Define the input layer
    input_img = tf.keras.Input(shape=input_shape)
    
    # CONV2D: 8 filters 4x4, stride of 1, padding 'SAME'
    #TODO
    
    # Apply ReLU activation function
    #TODO
    
    # MAXPOOL: window 8x8, stride 8, padding 'SAME'
    #TODO
    
    # CONV2D: 16 filters 2x2, stride 1, padding 'SAME'
    #TODO
    
    # Apply ReLU activation function
    #TODO
    
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    #TODO
    
    # Flatten
    #TODO
    
    # Dense layer
    # 6 neurons in output layer. Hint: one of the arguments should be "activation='softmax'" 
    #TODO
    
    # Define the Keras model
    model = tf.keras.Model(inputs=input_img, outputs=out)
    
    return model

In [ ]:
conv_model = convolutional_model((64, 64, 3))
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()
    
output = [['InputLayer', [(None, 64, 64, 3)], 0],
        ['Conv2D', (None, 64, 64, 8), 392, 'same', 'linear', 'GlorotUniform'],
        ['ReLU', (None, 64, 64, 8), 0],
        ['MaxPooling2D', (None, 8, 8, 8), 0, (8, 8), (8, 8), 'same'],
        ['Conv2D', (None, 8, 8, 16), 528, 'same', 'linear', 'GlorotUniform'],
        ['ReLU', (None, 8, 8, 16), 0],
        ['MaxPooling2D', (None, 2, 2, 16), 0, (4, 4), (4, 4), 'same'],
        ['Flatten', (None, 64), 0],
        ['Dense', (None, 6), 390, 'softmax']]
    
comparator(summary(conv_model), output)

Both the Sequential and Functional APIs return a TF Keras model object. The only difference is how inputs are handled inside the object model.

#### <font color='blue'> Q2.2.1 How to compute the number of parameters reported per layer? 

<a name='2.3'></a>
### <font color='darkviolet'> 2.3 - Train the Model

In [ ]:
# create train and test datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(64)

In [ ]:
# Print the size of the train_dataset
train_size = tf.data.experimental.cardinality(train_dataset).numpy()
print('Number of batches in train_dataset:', train_size)
print('Number of samples in train_dataset:', train_size * 64)

# Print the size of the test_dataset
test_size = tf.data.experimental.cardinality(test_dataset).numpy()
print('Number of batches in test_dataset:', test_size)
print('Number of samples in test_dataset:', test_size * 64)

In [ ]:
# Print the contents of the first batch of the train_dataset
for X, Y in train_dataset.take(1):
    print('Shape of input features (X):', X.shape)
    print('Shape of labels (Y):', Y.shape)
    print('Input features (X):', X)
    print('Labels (Y):', Y)

In [ ]:
# Compile the model
conv_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

# Train the model
history = conv_model.fit(train_dataset,
                          epochs=100,
                          validation_data=test_dataset,
                          verbose=2)

<a name='2.4'></a>
### <font color='darkviolet'> 2.4 - History

The history object is an output of the `.fit()` operation, and provides a record of all the loss and metric values in memory. It's stored as a dictionary that you can retrieve at `history.history`:

In [ ]:
# Create a DataFrame with the loss and accuracy values
df_loss_acc = pd.DataFrame(history.history)

# Extract the training and validation loss values and rename the columns
df_loss = df_loss_acc[['loss', 'val_loss']]
df_loss = df_loss.rename(columns={'loss': 'Training Loss', 'val_loss': 'Validation Loss'})

# Extract the training and validation accuracy values and rename the columns
df_acc = df_loss_acc[['accuracy', 'val_accuracy']]
df_acc = df_acc.rename(columns={'accuracy': 'Training Accuracy', 'val_accuracy': 'Validation Accuracy'})

# Plot the training and validation loss values
df_loss.plot(title='Model Loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')

# Plot the training and validation accuracy values
df_acc.plot(title='Model Accuracy', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')

#### <font color='blue'> Q2.4.1 Observe your training history and describe.

<a name='ex-3'></a>
### <font color='blue'> Exercise 3 - output volume computation
    
You have an input volume that is 32 x 32 x 16, and apply max pooling with a stride of 2 and a filter size of 2.
    
What is the output volume?